# DB Paris

In [34]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# read the data
df = pd.read_csv("db_paris.csv")
df["DatetimeOfCollection"] = pd.to_datetime(df["DateOfCollection"] + " " + df["TimeOfCollection"], format="%Y.%m.%d %H:%M")
df["TargetDate"] = pd.to_datetime(df["TargetDate"], format="%Y.%m.%d")
df["QueryDate"] = pd.to_datetime(df["DateOfCollection"])

# Overall average ticket price and standard deviation
mean_price = df["Price"].mean()
std_price = df["Price"].std()
print(f"Overall DB Paris (avg): CHF {mean_price:.2f}")
print(f"Overall DB Paris (sd): CHF {std_price:.2f}")

# Overall Price Distribution Histogram
fig2 = px.histogram(df, x="Price", nbins=20, marginal="box", title="Overall Price Distribution Histogram")
fig2.update_layout(xaxis_title="Price(CHF)", yaxis_title="Frequency")
fig2.show()

# Each departure date's average ticket price and sd
avg_std_price_by_target = (
    df.groupby("TargetDate")["Price"]
    .agg(AveragePrice="mean", PriceStd="std")
    .reset_index()
)
print("\nEach departure date's average ticket price and sd：")
print(avg_std_price_by_target)

# Each departure date's price change numbers
def count_price_changes(group):
    return (group["Price"].diff() != 0).sum()

change_counts = (
    df.sort_values(["TargetDate", "DatetimeOfCollection"])
      .groupby("TargetDate")
      .apply(count_price_changes)
      .reset_index(name="PriceChangeCount")
)
print("\nEach departure date's price change numbers：")
print(change_counts)

# Box Plot of Ticket Price Variation by Departure Date
fig3 = px.box(df, x="TargetDate", y="Price", points="all", title="Box Plot of Ticket Price Variation by Departure Date")
fig3.update_layout(xaxis_title="Departure Date", yaxis_title="Price (CHF)")
fig3.show()

# Ticket Price Variation from Day to Night
target_dates = sorted(df_filtered["TargetDate"].dropna().unique())

for target in target_dates:
    df_target = df_filtered[df_filtered["TargetDate"] == target].copy()

    fig = px.line(
        df_target,
        x="TimeOfCollection", 
        y="Price",
        color=df_target["QueryDate"].astype(str),
        markers=True,
        title=f"Ticket Price by Time of Day for {target.date()}",
        labels={
            "TimeOfCollection": "Time of Day",
            "Price": "Ticket Price (CHF)",
            "color": "Query Date"
        }
    )
    
    fig.update_layout(hovermode="x unified")
    fig.show()

Overall DB Paris (avg): CHF 145.31
Overall DB Paris (sd): CHF 41.43



Each departure date's average ticket price and sd：
  TargetDate  AveragePrice   PriceStd
0 2025-05-26    128.142857  14.631359
1 2025-06-04    109.423810  11.067951
2 2025-06-06    198.363095  19.803129

Each departure date's price change numbers：
  TargetDate  PriceChangeCount
0 2025-05-26                41
1 2025-06-04                42
2 2025-06-06                42


/var/folders/pv/hp1n3rgx60v9hl0srdnn0znc0000gn/T/ipykernel_44776/2993124771.py:38: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

